# Data Analysis with Generation Models

In this section, we utilize the graph-based generation models to detect the abnormal/malicious data in the log dataset. Generation models are able to learn the underlying data distribution of the normal data. When the model is trained on the normal dataset, if a normal data sample is input and forwared through the model, the reconstruction error after the backward propagation should keeps low. However, if the input is an abnormal data sample, the reconstruction error will be relatively much higher. Therefore, through analyzing the reconstruction error, we are able to detect the abnormal data points feeded into the model. We conducted the data analysis leveraging the Denoising Diffusion Probabilistic Model (DDPM), with the Variational Autoencoder (VAE) as the comparison benchmark. Both models are integrated with the Graph Transformer model to capture the graph structure information.

The training was conducted on 56,576 normal log graphs with the batch size of 32. Due to the data loeading and training process of generation models are time-consuming, we used sperate Python files to reconstruct the graph data, collate the data batches, train the models, and saved the trained model parameters. The specific code files and example tensor format are available in the GitHub repository [here](https://github.com/Klasnov/graph_abnormal_detect). In this report section, we will show the code snippets ***without*** executed outputs. The code content in this section is adapted from the tutorial code files, with modifications to our specific dataset application, like the explicit positional encoding of the graph structure based on the log sequence.


## Training Graph Reconstruction with DGL Library

We first load the networkx graph files after the previous data preprocessing. The graph data is then converted to Tensor format with DGL library, as DGL can help to easily extract and manipulate the node and edge features in the graph data.

In the networkx graph, the node features we are going to use are the node type in the computational system and the node log ID. Since some ID values are very frequent appearing in the log data, for each graph with specific size of 100 nodes, we reindex the node local ID, as the node positional encoding later, by random values in the range of 0 to 99. This may avoid the model to overfit on the specific global node ID values. The edge features are the operation type between the nodes.

In [ ]:
import os
import gzip
import networkx as nx
import dgl
import torch
import numpy as np

NUM_NODE = 100

TYPES = ["THREAD", "FILE", "REGISTRY", "FLOW", "USER_SESSION", "SERVICE", "PROCESS", "MODULE", "TASK", "SHELL"]
TYPE_MAP = {type: i for i, type in enumerate(TYPES)}

ACTIONS = [
    "FILE_CREATE", "FILE_DELETE", "FILE_MODIFY", "FILE_READ", "FILE_RENAME",
    "FILE_WRITE", "FLOW_MESSAGE", "FLOW_OPEN", "MODULE_LOAD", "PROCESS_CREATE",
    "PROCESS_OPEN", "PROCESS_TERMINATE", "REGISTRY_ADD", "REGISTRY_EDIT",
    "REGISTRY_REMOVE", "SERVICE_CREATE", "SHELL_COMMAND", "TASK_CREATE",
    "TASK_DELETE", "TASK_MODIFY", "TASK_START", "THREAD_CREATE", "THREAD_REMOTE_CREATE",
    "THREAD_TERMINATE", "USER_SESSION_GRANT", "USER_SESSION_INTERACTIVE","USER_SESSION_LOGIN",
    "USER_SESSION_LOGOUT", "USER_SESSION_REMOTE", "USER_SESSION_UNLOCK"
    ]
ACTION_MAP = {action: i for i, action in enumerate(ACTIONS)}

def nx_to_dgl(nx_graph):
    dgl_graph = dgl.graph(([], []))

    node_ids = list(nx_graph.nodes)
    dgl_graph.add_nodes(len(node_ids))
    nx_to_dgl_node_map = {node: i for i, node in enumerate(node_ids)}
    if len(node_ids) != NUM_NODE:
        raise ValueError("Number of nodes in graph is not 100")
    
    id_set = set()
    for node in node_ids:
        node_data = nx_graph.nodes[node]
        node_id = node_data.get("nodeID", -1)
        id_set.add(node_id)
    
    # Map nodeIDs to random integer in range of [0, 99], avoid overfitting to nodeIDs
    id_set_len = len(id_set)
    rand_ids = np.random.choice(100, id_set_len, replace=False)
    rand_id_Map = {node_id: rand_id for node_id, rand_id in zip(id_set, rand_ids)}
    
    # Map node features
    node_types = []
    ids = []
    
    for node in node_ids:
        node_data = nx_graph.nodes[node]
        node_type = node_data.get("node_type", "")
        node_id = node_data.get("nodeID", -1)
        node_types.append(TYPE_MAP.get(node_type, -1))
        ids.append(rand_id_Map.get(node_id, -1))
    
    dgl_graph.ndata["type"] = torch.tensor(node_types, dtype=torch.float32)
    dgl_graph.ndata["id"] = torch.tensor(ids, dtype=torch.float32)
    
    # Map edge features
    edges = list(nx_graph.edges(data=True))
    src_nodes = []
    dst_nodes = []
    edge_actions = []
    
    for edge in edges:
        src, dst, edge_data = edge
        src_nodes.append(nx_to_dgl_node_map[src])
        dst_nodes.append(nx_to_dgl_node_map[dst])
        edge_actions.append(ACTION_MAP.get(edge_data.get("action", ""), -1))
    
    dgl_graph.add_edges(src_nodes, dst_nodes)
    dgl_graph.edata["action"] = torch.tensor(edge_actions, dtype=torch.float32)
    
    return dgl_graph



dir_path = "data/train_graphs/"
if not os.path.exists(dir_path):
    raise ValueError("The path does not exist")

train_graph_list = []
for chid_dirs in os.listdir(dir_path):
    chid_dirs_path = os.path.join(dir_path, chid_dirs)
    if not os.path.isdir(chid_dirs_path):
        continue
    for file in os.listdir(chid_dirs_path):
        if file.endswith(".gz"):
            file_path = os.path.join(chid_dirs_path, file)
            with gzip.open(file_path, "rb") as f:
                nx_graph = nx.read_gml(f)
                dgl_graph = nx_to_dgl(nx_graph)
                train_graph_list.append(dgl_graph)

## Data Batching and Storage in Tensor Format

To make the training process more efficient, we collate the graph data into batches with batch size of 32. The batched graph data is saved in the PyTorch tensor format, which can be directly loaded in the training process.

In this process, the edge features are converted from the coordinate sparse matrix format (COO) into the dense weighted adjacency matrix format.

In [ ]:
import os
import torch
from math import ceil


BATCH_SIZE = 32


def get_weighted_adjacency_matrix(graph):
    adj_spr = graph.adjacency_matrix(scipy_fmt="coo")
    edge_freq = graph.edata["action"]
    num_nodes = graph.number_of_nodes()
    adj = torch.zeros(num_nodes, num_nodes)
    for i, j, freq in zip(adj_spr.row, adj_spr.col, edge_freq):
        adj[i, j] = freq
    adj = adj + adj.T
    return adj


def batch_save(graph_list, path, index, batch_size=32):
    if len(graph_list) < batch_size:
        batch_size = len(graph_list)
    node_type_list = []
    node_id_list = []
    edge_action_list = []
    for graph in graph_list:
        node_type_list.append(graph.ndata["type"])
        node_id_list.append(graph.ndata["id"])
        adj = get_weighted_adjacency_matrix(graph)
        edge_action_list.append(adj)
    h = torch.cat(node_type_list, dim=0).view(batch_size, 100).long()
    pe = torch.cat(node_id_list, dim=0).view(batch_size, 100).long()
    e = torch.cat(edge_action_list, dim=0).view(batch_size, 100, 100).long()
    torch.save(h, path + "h_" + str(index) + ".pt")
    torch.save(pe, path + "pe_" + str(index) + ".pt")
    torch.save(e, path + "e_" + str(index) + ".pt")
    return h, pe, e


def batch_graph(graph_list=None, batch_size=32, path="data/train/", index=0, load=True):
    if not load:
        if not os.path.exists(path):
            os.makedirs(path)
        num_batch = ceil(len(graph_list) / batch_size)
        for i in range(num_batch):
            end_batch_index = (i + 1) * batch_size
            if end_batch_index > len(graph_list):
                end_batch_index = len(graph_list)
            batch_graph_list = graph_list[i * batch_size: end_batch_index]
            save_path = path + "batch_" + str(i) + "/"
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            batch_save(batch_graph_list, save_path, i)
            h = None; pe = None; e = None
    else:
        save_path = path + "batch_" + str(index) + "/"
        if not os.path.exists(save_path):
            raise ValueError("The path does not exist")
        h = torch.load(save_path + "h_" + str(index) + ".pt", weights_only=True)
        pe = torch.load(save_path + "pe_" + str(index) + ".pt", weights_only=True)
        e = torch.load(save_path + "e_" + str(index) + ".pt", weights_only=True)
    return h, pe, e


_, _, _ = batch_graph(train_graph_list, batch_size=BATCH_SIZE, load=False)

## Training Script for DDPM Model

The training script for the Denoising Diffusion Probabilistic Model (DDPM) is shown below. The model is trained with the batched graph data, and the trained model parameters are saved for the later reconstruction and analysis process.

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
NUM_NODE = 100
TYPES = ["THREAD", "FILE", "REGISTRY", "FLOW", "USER_SESSION", "SERVICE", "PROCESS", "MODULE", "TASK", "SHELL"]
ACTIONS = [
    "FILE_CREATE", "FILE_DELETE", "FILE_MODIFY", "FILE_READ", "FILE_RENAME", "FILE_WRITE", "FLOW_MESSAGE",
    "FLOW_OPEN", "MODULE_LOAD", "PROCESS_CREATE", "PROCESS_OPEN", "PROCESS_TERMINATE", "REGISTRY_ADD",
    "REGISTRY_EDIT", "REGISTRY_REMOVE", "SERVICE_CREATE", "SHELL_COMMAND", "TASK_CREATE", "TASK_DELETE",
    "TASK_MODIFY", "TASK_START", "THREAD_CREATE", "THREAD_REMOTE_CREATE", "THREAD_TERMINATE", "USER_SESSION_GRANT",
    "USER_SESSION_INTERACTIVE","USER_SESSION_LOGIN", "USER_SESSION_LOGOUT", "USER_SESSION_REMOTE", "USER_SESSION_UNLOCK"
]
NUM_BATCH = 1768
NUM_GRAPH = NUM_BATCH * BATCH_SIZE
NUM_WARMUP = 2 * max(NUM_NODE, NUM_GRAPH // BATCH_SIZE)

EPOCH_NUM = 1000
INIT_LR = 0.0003


def get_weighted_adjacency_matrix(graph):
    adj_spr = graph.adjacency_matrix(scipy_fmt="coo")
    edge_freq = graph.edata["action"]
    num_nodes = graph.number_of_nodes()
    adj = torch.zeros(num_nodes, num_nodes)
    for i, j, freq in zip(adj_spr.row, adj_spr.col, edge_freq):
        adj[i, j] = freq
    adj = adj + adj.T
    return adj


def get_batch(path="data/train/", index=0):
    save_path = path + "batch_" + str(index) + "/"
    if not os.path.exists(save_path):
        raise ValueError("The path does not exist")
    h = torch.load(save_path + "h_" + str(index) + ".pt", weights_only=True)
    pe = torch.load(save_path + "pe_" + str(index) + ".pt", weights_only=True)
    e = torch.load(save_path + "e_" + str(index) + ".pt", weights_only=True)
    return h, pe, e


def sym_tensor(x):
    x = x.permute(0, 3, 1, 2) # [bs, n, n, d]
    triu = torch.triu(x,diagonal=1).transpose(3,2) # [bs, d, n, n]
    mask = (triu.abs()>0).float()                  # [bs, d, n, n]
    x =  x * (1 - mask ) + mask * triu             # [bs, d, n, n]
    x = x.permute(0, 2, 3, 1) # [bs, n, n, d]
    return x               # [bs, n, n, d]

class Embed_G(nn.Module):
    def __init__(self, params):
        super().__init__()
        d = params["d"]
        self.Embed_h = nn.Embedding(params["num_type"], d)
        self.Embed_e = nn.Embedding(params["num_action"], d)
        self.Embed_pe = nn.Embedding(params["num_node"], d)
    
    def forward(self, h, e, pe):
        pe = self.Embed_pe(pe) # [bs, n, d]
        h = self.Embed_h(h)
        h = h + pe
        e = self.Embed_e(e) # [bs, n, n, d]
        e = e + pe.unsqueeze(1) + pe.unsqueeze(2)
        e = sym_tensor(e)
        return h, e


class Attention_Layer(nn.Module):
    def __init__(self, d, d_head, drop):
        super().__init__()
        self.Q = nn.Linear(d, d_head)
        self.K = nn.Linear(d, d_head)
        self.V = nn.Linear(d, d_head)
        self.E = nn.Linear(d, d_head)
        self.Ni = nn.Linear(d, d_head)
        self.Nj = nn.Linear(d, d_head)
        self.Drop_Att = nn.Dropout(drop)
        self.sqrt_d = torch.sqrt(torch.tensor(d_head))
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        Q = self.Q(h) # [bs, n, d_head]
        K = self.K(h)
        V = self.V(h)
        Q = Q.unsqueeze(2)  # [bs, n, 1, d_head]
        K = K.unsqueeze(1)  # [bs, 1, n, d_head]
        E = self.E(e)       # [bs, n, n, d_head]
        Ni = self.Ni(h).unsqueeze(2) # [bs, n, 1, d_head]
        Nj = self.Nj(h).unsqueeze(1) # [bs, 1, n, d_head]
        e = E + Ni + Nj              # [bs, n, n, d_head]
        Att = (Q * e * K).sum(dim=-1) / self.sqrt_d # [bs, n, n]
        Att = torch.softmax(Att, dim=1)             # [bs, n, n]
        Att = self.Drop_Att(Att)
        h = Att @ V # [bs, n, d_head]
        return h, e


class MAH_Layer(nn.Module):
    def __init__(self, d, head_num, drop):
        super().__init__()
        d_head = d // head_num
        self.heads = nn.ModuleList([Attention_Layer(d, d_head, drop) for _ in range(head_num)])
        self.WO_h = nn.Linear(d, d)
        self.WO_e = nn.Linear(d, d)
        self.Drop_h = nn.Dropout(drop)
        self.Drop_e = nn.Dropout(drop)
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        h_MHA = []
        e_MHA = []
        for head in self.heads:
            h_mha, e_mha = head(h, e)
            h_MHA.append(h_mha)
            e_MHA.append(e_mha)
        h = self.Drop_h(self.WO_h(torch.cat(h_MHA, dim=2)))
        e = self.Drop_e(self.WO_e(torch.cat(e_MHA, dim=3)))
        return h, e


class GT_Layer(nn.Module):
    def __init__(self, d, num_head, drop):
        super().__init__()
        self.Norm_h_1 = nn.LayerNorm(d)
        self.Norm_e_1 = nn.LayerNorm(d)
        self.MHA = MAH_Layer(d, num_head, drop)
        self.Norm_h_2 = nn.LayerNorm(d)
        self.Norm_e_2 = nn.LayerNorm(d)
        self.MLP_h = nn.Sequential(nn.Linear(d, 4*d), nn.ReLU(), nn.Linear(4*d, d))
        self.MLP_e = nn.Sequential(nn.Linear(d, 4*d), nn.ReLU(), nn.Linear(4*d, d))
        self.Drop_h = nn.Dropout(drop)
        self.Drop_e = nn.Dropout(drop)
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        h = self.Norm_h_1(h)
        e = self.Norm_e_1(e)
        h_MHA, e_MHA = self.MHA(h, e)
        h = h + h_MHA
        h = h + self.MLP_h(self.Norm_h_2(h))
        e = e + e_MHA
        e = e + self.MLP_e(self.Norm_e_2(e))
        h = self.Drop_h(h)
        e = self.Drop_e(e)
        return h, e


class UNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        d = params["d"]
        # Graph Embedding
        self.num_node = params["num_node"]
        self.num_t = params["num_t"]
        self.num_type = params["num_type"]
        self.num_action = params["num_action"]
        self.Embed_h = nn.Linear(self.num_type, d)
        self.Embed_e = nn.Linear(self.num_action, d)
        self.pe_h = nn.Embedding(self.num_node, d)
        # Time Embedding
        self.pe_t = nn.Sequential(nn.Embedding(self.num_t, d), nn.ReLU(), nn.Linear(d, d))
        # GT Layers
        num_gt_layer = params["num_gt_layer"]
        num_head = params["num_head"]
        drop = params["drop"]
        self.GT_Layers = nn.ModuleList([GT_Layer(d, num_head, drop) for _ in range(num_gt_layer)])
        # Output Layer
        self.LN_h = nn.Linear(d, self.num_type)
        self.LN_e = nn.Linear(d, self.num_action)
    
    def forward(self, h, e, pe_h, sample_t):
        # Embedding for Graph
        pe_h = self.pe_h(pe_h) # [bs, n, d]
        h_t = self.Embed_h(h)
        h_t = h_t + pe_h
        e_t = self.Embed_e(e) # [bs, n, n, d]
        e_t = e_t + pe_h.unsqueeze(1)
        e_t = sym_tensor(e_t)
        # Embedding for Time
        pe_t = self.pe_t(sample_t) # [bs, d]
        # GT Layers
        for GT_Layer in self.GT_Layers:
            h_t = h_t + pe_t.unsqueeze(1) # [bs, n, d]
            e_t = e_t + pe_t.unsqueeze(1).unsqueeze(2) # [bs, n, n, d]
            h_t, e_t = GT_Layer(h_t, e_t)
            e_t = sym_tensor(e_t)
        # Output Layer
        h_t_minus_one = self.LN_h(h_t)
        e_t_minus_one = self.LN_e(e_t)
        return h_t_minus_one, e_t_minus_one
    

class DDPM(nn.Module):
    def __init__(self, num_t, beta_1, beta_t, params):
        super().__init__()
        self.device = params["device"]
        self.num_type = params["num_type"]
        self.num_action = params["num_action"]
        self.UNet = UNet(params)
        self.num_t = num_t
        self.alpha_t = 1.0 - torch.linspace(beta_1, beta_t, num_t).to(self.device)
        self.alpha_bar_t = torch.cumprod(self.alpha_t, dim=0)
    
    def forward(self, h_0, e_0, sample_t, noise_h0, noise_e0):
        h0 = torch.nn.functional.one_hot(h_0, self.num_type).float()
        e0 = torch.nn.functional.one_hot(e_0, self.num_action).float()
        bs = len(sample_t)
        sqrt_alpha_bar_t = torch.sqrt(self.alpha_bar_t[sample_t])
        sqrt_one_minus_alpha_bar_t = torch.sqrt(1.0 - self.alpha_bar_t[sample_t])
        h_t = sqrt_alpha_bar_t.view(bs, 1, 1) * h0 + sqrt_one_minus_alpha_bar_t.view(bs, 1, 1) * noise_h0
        e_t = sqrt_alpha_bar_t.view(bs, 1, 1, 1) * e0 + sqrt_one_minus_alpha_bar_t.view(bs, 1, 1, 1) * noise_e0
        return h_t, e_t
    
    def backward(self, h_t, e_t, pe_h, sample_t):
        noise_pred_h_t, noise_pred_e_t = self.UNet(h_t, e_t, pe_h, sample_t)
        return noise_pred_h_t, noise_pred_e_t
    

def train_ddpm(num_t, beta_1, beta_t, net_params, load_save=True, model_path="model/ddpm/"):
    torch.random.manual_seed(0)
    ddpm = DDPM(num_t, beta_1, beta_t, net_params).to(DEVICE)
    if load_save:
        if os.path.exists(model_path + "ddpm.pt"):
            ddpm.load_state_dict(torch.load(model_path + "ddpm.pt", weights_only=True))
    
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    init_lr = INIT_LR
    optimizer = torch.optim.AdamW(ddpm.parameters(), lr=init_lr)
    scheduler_warmup = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda t: min((t+1)/NUM_WARMUP, 1.0))
    scheduler_tracker = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.95, patience=1)

    num_epoch = EPOCH_NUM
    num_warmup_batch = 0

    train_loss_drop_patience = 5
    loss_dropping = True
    train_loss_drop_cnt = 0
    previous_best_loss = float("inf")

    start = time.time()
    for epoch in range(num_epoch):
        running_loss = 0.0
        num_batch = 0

        ddpm.train()

        for i in range(NUM_BATCH):
            h, pe, e = get_batch(index=i)
            h = h.to(DEVICE)
            pe = pe.to(DEVICE)
            e = e.to(DEVICE)
            batch_sample_t = torch.randint(0, num_t, (BATCH_SIZE,)).long().to(DEVICE)
            batch_noise_h_t = torch.randn(BATCH_SIZE, NUM_NODE, len(TYPES)).to(DEVICE)
            batch_noise_e_t = torch.randn(BATCH_SIZE, NUM_NODE, NUM_NODE, len(ACTIONS)).to(DEVICE)
            batch_noise_e_t = sym_tensor(batch_noise_e_t)

            h_t, e_t = ddpm(h, e, batch_sample_t, batch_noise_h_t, batch_noise_e_t)
            noise_pred_h_t, noise_pred_e_t = ddpm.backward(h_t, e_t, pe, batch_sample_t)

            loss = torch.nn.MSELoss()(noise_pred_h_t, batch_noise_h_t) + torch.nn.MSELoss()(noise_pred_e_t, batch_noise_e_t)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if num_batch < NUM_WARMUP:
                scheduler_warmup.step()
            num_batch += 1

            running_loss += loss.detach().item()
            num_batch += 1

            del h, pe, e, batch_sample_t, batch_noise_h_t, batch_noise_e_t, h_t, e_t, noise_pred_h_t, noise_pred_e_t
            torch.cuda.empty_cache()
        
        mean_loss = running_loss / num_batch
        if num_warmup_batch >= NUM_WARMUP:
            scheduler_tracker.step(mean_loss)
        elapsed = (time.time() - start) / 60
        print(f"Epoch {epoch+1}/{num_epoch}  Loss: {mean_loss:.6f}  lr: {optimizer.param_groups[0]['lr']:.6f}  Time: {elapsed:.2f} mins")

        if optimizer.param_groups[0]['lr'] < 1e-5:
            print("Early stopping since lr < 1e-5")
            break

        if mean_loss < previous_best_loss:
            previous_best_loss = mean_loss
            train_loss_drop_cnt = 0
            loss_dropping = True
        else:
            train_loss_drop_cnt += 1
            loss_dropping = False
            if train_loss_drop_cnt >= train_loss_drop_patience:
                print("Early stopping since loss is not dropping")
                break

        if (epoch + 1) % 5 == 0 and loss_dropping:
            torch.save(ddpm.state_dict(), model_path + "ddpm.pt")
    
    torch.save(ddpm.state_dict(), model_path + "ddpm.pt")
    print(f"Finished the training of DDPM, with the best loss {previous_best_loss:.6f}, and the total time {elapsed:.2f} mins")


if __name__ == "__main__":
    beta_1 = 0.0001
    beta_t = 0.1
    num_t = 200

    ddpm_net_params = {
        "num_type": len(TYPES),
        "num_action": len(ACTIONS),
        "num_node": NUM_NODE,
        "num_gt_layer": 6,
        "num_head": 4,
        "d": 32 * 4,
        "num_t": num_t,
        "drop": 0,
        "device": DEVICE
    }

    train_ddpm(num_t, beta_1, beta_t, ddpm_net_params)

## Training Script for VAE Model

The training script for the Variational Autoencoder (VAE) model is shown below. The model is trained with the batched graph data, and the trained model parameters are saved for the later reconstruction and analysis process.

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
NUM_NODE = 100
TYPES = ["THREAD", "FILE", "REGISTRY", "FLOW", "USER_SESSION", "SERVICE", "PROCESS", "MODULE", "TASK", "SHELL"]
ACTIONS = [
    "FILE_CREATE", "FILE_DELETE", "FILE_MODIFY", "FILE_READ", "FILE_RENAME", "FILE_WRITE", "FLOW_MESSAGE",
    "FLOW_OPEN", "MODULE_LOAD", "PROCESS_CREATE", "PROCESS_OPEN", "PROCESS_TERMINATE", "REGISTRY_ADD",
    "REGISTRY_EDIT", "REGISTRY_REMOVE", "SERVICE_CREATE", "SHELL_COMMAND", "TASK_CREATE", "TASK_DELETE",
    "TASK_MODIFY", "TASK_START", "THREAD_CREATE", "THREAD_REMOTE_CREATE", "THREAD_TERMINATE", "USER_SESSION_GRANT",
    "USER_SESSION_INTERACTIVE","USER_SESSION_LOGIN", "USER_SESSION_LOGOUT", "USER_SESSION_REMOTE", "USER_SESSION_UNLOCK"
]
NUM_BATCH = 1768
NUM_GRAPH = NUM_BATCH * BATCH_SIZE
NUM_WARMUP = 2 * max(NUM_NODE, NUM_GRAPH // BATCH_SIZE)

EPOCH_NUM = 1000
INIT_LR = 0.0003


def get_weighted_adjacency_matrix(graph):
    adj_spr = graph.adjacency_matrix(scipy_fmt="coo")
    edge_freq = graph.edata["action"]
    num_nodes = graph.number_of_nodes()
    adj = torch.zeros(num_nodes, num_nodes)
    for i, j, freq in zip(adj_spr.row, adj_spr.col, edge_freq):
        adj[i, j] = freq
    adj = adj + adj.T
    return adj


def get_batch(path="data/train/", index=0):
    save_path = path + "batch_" + str(index) + "/"
    if not os.path.exists(save_path):
        raise ValueError("The path does not exist")
    h = torch.load(save_path + "h_" + str(index) + ".pt", weights_only=True)
    pe = torch.load(save_path + "pe_" + str(index) + ".pt", weights_only=True)
    e = torch.load(save_path + "e_" + str(index) + ".pt", weights_only=True)
    return h, pe, e


def sym_tensor(x):
    x = x.permute(0, 3, 1, 2) # [bs, n, n, d]
    triu = torch.triu(x,diagonal=1).transpose(3,2) # [bs, d, n, n]
    mask = (triu.abs()>0).float()                  # [bs, d, n, n]
    x =  x * (1 - mask ) + mask * triu             # [bs, d, n, n]
    x = x.permute(0, 2, 3, 1) # [bs, n, n, d]
    return x               # [bs, n, n, d]

class Embed_G(nn.Module):
    def __init__(self, params):
        super().__init__()
        d = params["d"]
        self.Embed_h = nn.Embedding(params["num_type"], d)
        self.Embed_e = nn.Embedding(params["num_action"], d)
        self.Embed_pe = nn.Embedding(params["num_node"], d)
    
    def forward(self, h, e, pe):
        pe = self.Embed_pe(pe) # [bs, n, d]
        h = self.Embed_h(h)
        h = h + pe
        e = self.Embed_e(e) # [bs, n, n, d]
        e = e + pe.unsqueeze(1)
        e = sym_tensor(e)
        return h, e


class Attention_Layer(nn.Module):
    def __init__(self, d, d_head, drop):
        super().__init__()
        self.Q = nn.Linear(d, d_head)
        self.K = nn.Linear(d, d_head)
        self.V = nn.Linear(d, d_head)
        self.E = nn.Linear(d, d_head)
        self.Ni = nn.Linear(d, d_head)
        self.Nj = nn.Linear(d, d_head)
        self.Drop_Att = nn.Dropout(drop)
        self.sqrt_d = torch.sqrt(torch.tensor(d_head))
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        Q = self.Q(h) # [bs, n, d_head]
        K = self.K(h)
        V = self.V(h)
        Q = Q.unsqueeze(2)  # [bs, n, 1, d_head]
        K = K.unsqueeze(1)  # [bs, 1, n, d_head]
        E = self.E(e)       # [bs, n, n, d_head]
        Ni = self.Ni(h).unsqueeze(2) # [bs, n, 1, d_head]
        Nj = self.Nj(h).unsqueeze(1) # [bs, 1, n, d_head]
        e = E + Ni + Nj              # [bs, n, n, d_head]
        Att = (Q * e * K).sum(dim=-1) / self.sqrt_d # [bs, n, n]
        Att = torch.softmax(Att, dim=1)             # [bs, n, n]
        Att = self.Drop_Att(Att)
        h = Att @ V # [bs, n, d_head]
        return h, e


class MAH_Layer(nn.Module):
    def __init__(self, d, head_num, drop):
        super().__init__()
        d_head = d // head_num
        self.heads = nn.ModuleList([Attention_Layer(d, d_head, drop) for _ in range(head_num)])
        self.WO_h = nn.Linear(d, d)
        self.WO_e = nn.Linear(d, d)
        self.Drop_h = nn.Dropout(drop)
        self.Drop_e = nn.Dropout(drop)
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        h_MHA = []
        e_MHA = []
        for head in self.heads:
            h_mha, e_mha = head(h, e)
            h_MHA.append(h_mha)
            e_MHA.append(e_mha)
        h = self.Drop_h(self.WO_h(torch.cat(h_MHA, dim=2)))
        e = self.Drop_e(self.WO_e(torch.cat(e_MHA, dim=3)))
        return h, e


class GT_Layer(nn.Module):
    def __init__(self, d, num_head, drop):
        super().__init__()
        self.Norm_h_1 = nn.LayerNorm(d)
        self.Norm_e_1 = nn.LayerNorm(d)
        self.MHA = MAH_Layer(d, num_head, drop)
        self.Norm_h_2 = nn.LayerNorm(d)
        self.Norm_e_2 = nn.LayerNorm(d)
        self.MLP_h = nn.Sequential(nn.Linear(d, 4*d), nn.ReLU(), nn.Linear(4*d, d))
        self.MLP_e = nn.Sequential(nn.Linear(d, 4*d), nn.ReLU(), nn.Linear(4*d, d))
        self.Drop_h = nn.Dropout(drop)
        self.Drop_e = nn.Dropout(drop)
    
    def forward(self, h, e):
        # h: [bs, n, d]; e: [bs, n, n, d]
        h = self.Norm_h_1(h)
        e = self.Norm_e_1(e)
        h_MHA, e_MHA = self.MHA(h, e)
        h = h + h_MHA
        h = h + self.MLP_h(self.Norm_h_2(h))
        e = e + e_MHA
        e = e + self.MLP_e(self.Norm_e_2(e))
        h = self.Drop_h(h)
        e = self.Drop_e(e)
        return h, e


class VAE(nn.Module):
    def __init__(self, params):
        super().__init__()
        d = params["d"]
        
        # Graph Embedding
        self.num_node = params["num_node"]
        self.Embed_he = Embed_G(params)
        self.Embed_pe = nn.Embedding(params["num_node"], d)

        # GT Layers
        num_enc_layer = params["num_enc_layer"]
        num_dec_layer = params["num_dec_layer"]
        num_head = params["num_head"]
        drop = params["drop"]
        self.Enc_Layers = nn.ModuleList([GT_Layer(d, num_head, drop) for _ in range(num_enc_layer)])
        self.Dec_Layers = nn.ModuleList([GT_Layer(d, num_head, drop) for _ in range(num_dec_layer)])

        # Encoder
        dz = params["dz"]
        self.LN_q_mu = nn.Linear(d, dz)
        self.LN_q_logvar = nn.Linear(d, dz)

        # Decoder
        self.LN_p = nn.Linear(dz, d)

        # Output Layer
        self.Norm_Out_h = nn.LayerNorm(d)
        self.Norm_Out_e = nn.LayerNorm(d)
        self.LN_h = nn.Linear(d, params["num_type"])
        self.LN_e = nn.Linear(d, params["num_action"])
    
    def forward(self, h, e, pe, num_node=None):
        if num_node is None:
            num_node = self.num_node

        # Embedding
        h, e = self.Embed_he(h, e, pe)
        n = h.size(1)
        pe = self.Embed_pe(pe)
        # Encoder
        for Enc_Layer in self.Enc_Layers:
            h, e = Enc_Layer(h, e)
            e = sym_tensor(e)
        graph_token = h.mean(dim=1)
        q_mu = self.LN_q_mu(graph_token)
        q_logvar = self.LN_q_logvar(graph_token)
        q_std = torch.exp(q_logvar / 2)
        eps = torch.randn_like(q_std)
        z = q_mu + eps * q_std # [bs, dz]
        n = h.size(1)

        # Decoder
        z = self.LN_p(z) # [bs, d]
        h = z.unsqueeze(1).repeat(1, n, 1) # [bs, n, d]
        h = h + pe
        e = z.unsqueeze(1).unsqueeze(1).repeat(1, n, n, 1) # [bs, n, n, d]
        e = e + pe.unsqueeze(1) + pe.unsqueeze(2)
        e = sym_tensor(e)
        for Dec_Layer in self.Dec_Layers:
            h, e = Dec_Layer(h, e)
            e = sym_tensor(e)
        h = self.Norm_Out_h(h)
        e = self.Norm_Out_e(e)
        h = self.LN_h(h)
        e = self.LN_e(e)
        return h, e, q_mu, q_logvar
    

def train_vae(net_params, load_save=True, model_path="model/vae/"):
    torch.random.manual_seed(0)
    vae = VAE(net_params).to(DEVICE)
    if load_save:
        if os.path.exists(model_path + "vae.pt"):
            vae.load_state_dict(torch.load(model_path + "vae.pt", weights_only=True))
    
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    init_lr = INIT_LR
    optimizer = torch.optim.AdamW(vae.parameters(), lr=init_lr)
    scheduler_warmup = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda t: min((t+1)/NUM_WARMUP, 1.0))
    scheduler_tracker = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.95, patience=1)

    num_epoch = EPOCH_NUM
    num_warmup_batch = 0

    train_loss_drop_patience = 5
    loss_dropping = True
    train_loss_drop_cnt = 0
    previous_best_loss = float("inf")

    start = time.time()
    for epoch in range(num_epoch):
        running_loss = 0.0
        num_batch = 0

        vae.train()

        for i in range(NUM_BATCH):
            h, pe, e = get_batch(index=i)
            h = h.to(DEVICE)
            pe = pe.to(DEVICE)
            e = e.to(DEVICE)
            
            pred_h, pred_e, q_mu, q_logvar = vae(h, e, pe)
            loss_data = torch.nn.CrossEntropyLoss()(pred_h.view(BATCH_SIZE * NUM_NODE, len(TYPES)), h.view(BATCH_SIZE * NUM_NODE)) 
            loss_data += torch.nn.CrossEntropyLoss()(pred_e.view(BATCH_SIZE * NUM_NODE * NUM_NODE, len(ACTIONS)), e.view(BATCH_SIZE * NUM_NODE * NUM_NODE))
            loss_kl = -0.5 * torch.sum(1 + q_logvar - q_mu.pow(2) - q_logvar.exp())
            loss = 2.5 * loss_data + loss_kl
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(vae.parameters(), 0.25)
            optimizer.step()

            if num_batch < NUM_WARMUP:
                scheduler_warmup.step()
            num_batch += 1

            running_loss += loss.detach().item()
            num_batch += 1

            del h, pe, e, pred_h, pred_e, q_mu, q_logvar, loss_data, loss_kl, loss
            torch.cuda.empty_cache()
        
        mean_loss = running_loss / num_batch
        if num_warmup_batch >= NUM_WARMUP:
            scheduler_tracker.step(mean_loss)
        elapsed = (time.time() - start) / 60
        print(f"Epoch {epoch+1}/{num_epoch}  Loss: {mean_loss:.6f}  lr: {optimizer.param_groups[0]['lr']:.6f}  Time: {elapsed:.2f} mins")


        if optimizer.param_groups[0]['lr'] < 1e-6:
            print("Early stopping")
            break

        if mean_loss < previous_best_loss:
            previous_best_loss = mean_loss
            train_loss_drop_cnt = 0
            loss_dropping = True
        else:
            train_loss_drop_cnt += 1
            loss_dropping = False
            if train_loss_drop_cnt >= train_loss_drop_patience:
                print("Early stopping since loss is not dropping")
                break

        if (epoch + 1) % 5 == 0 and loss_dropping:
            torch.save(vae.state_dict(), model_path + "vae.pt")
    
    torch.save(vae.state_dict(), model_path + "vae.pt")
    print(f"Finished the training of VAE, with the best loss {previous_best_loss:.6f}, and the total time {elapsed:.2f} mins")


if __name__ == "__main__":
    vae_net_params = {
        "num_type": len(TYPES),
        "num_action": len(ACTIONS),
        "num_node": NUM_NODE,
        "num_enc_layer": 4,
        "num_dec_layer": 4,
        "num_head": 8,
        "drop": 0,
        "d": 16 * 8,
        "dz": 32
    }

    train_vae(vae_net_params)

> The seperated training code Python files, example batches of graph data in tensor format, and the trained model parameters are available in the GitHub repository [here](https://github.com/Klasnov/graph_abnormal_detect).

# Numerical Results

In this section, we will evaluate the trained DDPM and VAE models on the normal and abnormal log data. The reconstruction error of the models will be calculated and analyzed to detect the abnormal data points in the log dataset.

The analysis here is based on the assumption that data points following the Gaussian distribution. Based on the random subset of the normal training data, we leveraged its mean $\mu$ and standard deviation $\sigma$ to calculate the possibility $f(x)=\frac{1}{\sqrt{2\pi{\sigma}^2}}e^{-\frac{(x-\mu)^2}{2{\sigma}^2}}$ of the unseen data point $x$ being normal or abnormal.


## Random Subset of Normal Training Data

A random subset of the normal training data is selected to calculate the mean and standard deviation of the reconstruction error. The subset contains 10% of the normal training data, which is consisted of 177 training batches.

In [ ]:
import torch
import os


MAX_BATCH = 1768
NUM_BATCH = 177


def get_batch(path="data/train/", index=0):
    save_path = path + "batch_" + str(index) + "/"
    if not os.path.exists(save_path):
        raise ValueError(f"The path does not exist: {save_path}")
    h = torch.load(save_path + "h_" + str(index) + ".pt", weights_only=True)
    pe = torch.load(save_path + "pe_" + str(index) + ".pt", weights_only=True)
    e = torch.load(save_path + "e_" + str(index) + ".pt", weights_only=True)
    return h, pe, e


torch.random.manual_seed(0)
rand_indices = torch.randperm(MAX_BATCH)
pick_indices = rand_indices[:NUM_BATCH]
pick_indices = pick_indices.tolist()
for i in range(NUM_BATCH):
    h, pe, e = get_batch(index=pick_indices[i])
    if i == 0:
        h_all = h
        pe_all = pe
        e_all = e
    else:
        h_all = torch.cat((h_all, h), dim=0)
        pe_all = torch.cat((pe_all, pe), dim=0)
        e_all = torch.cat((e_all, e), dim=0)